## 📏 Real-Time Object Measurement

### ✅ If the Camera Never Moves
If the **camera position, angle, focus, and distance stay the same**, then the measurement remains accurate **only if the new object is placed exactly where the calibrated object was**.

---

### ✅ When It **Will Still Be Accurate**
* ✔️ Object is on the **same flat surface**
* ✔️ Object is at the **same distance from the camera**
* ✔️ Object is **not tilted** (same orientation as the remote)
* ✔️ Camera does **not auto-zoom or change focal length**
* ✔️ Lighting does not break contour detection

🔹 In this scenario, replacing the remote with an iPhone will still produce correct measurement.

---

### ❌ When It Becomes Inaccurate (Even if Camera Doesn’t Move)
| Issue                                 | Why it breaks                     |
| ------------------------------------- | --------------------------------- |
| Object is closer/farther from camera  | Pixel size changes                |
| Object is tilted upward/downward      | Perspective distortion            |
| Camera autofocus changes focal length | Scale becomes invalid             |
| Different edge detection shape        | Bounding box changes              |
| Object is thick or curved             | Detected contour is not true size |

---

### 📌 Key Reminder
Your current method uses **2D pixel-to-cm scaling** based on a **single object calibration**.
It assumes the object lies on **the same plane (same Z-depth)**.

```
cm_per_pixel = real_size_cm / detected_pixel_size
```

If depth changes → pixel size changes → result becomes wrong.

---

### 🔍 Example
| Object               | Distance | Pixel Height | Real Height | Result             |
| -------------------- | -------- | ------------ | ----------- | ------------------ |
| Remote (calibration) | 40 cm    | 250 px       | 16 cm       | ✅ Correct          |
| iPhone in same spot  | 40 cm    | 400 px       | 14.7 cm     | ✅ Correct          |
| iPhone moved closer  | 35 cm    | 450 px       | 14.7 cm     | ❌ Measured too big |

---

### ✅ Final Answer
* ✅ If the new object is in **exact same position** → measurement stays accurate.
* ❌ If distance or angle changes even slightly → measurement becomes inaccurate.

---

### 🛠 Want Accuracy for Any Object, Any Distance?
| Method                                   | Depth Safe? | Works for Any Object? |
| ---------------------------------------- | ----------- | --------------------- |
| Use reference object (coin, credit card) | ✅           | ✅                     |
| Use ArUco marker                         | ✅           | ✅                     |
| Stereo / depth camera                    | ✅           | ✅                     |
| Full camera calibration                  | ✅           | ✅                     |
| Your current single-object calibration   | ❌           | ❌                     |

## Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Real Time Object Detection using Real-Life References Measured Object

In [36]:
# === DISPLAY FUNCTION (unchanged) ===
def display_comparison(original_img, edge_img, title1="Edge Detection", title2="Detected Object"):
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))
    axs[0].imshow(cv2.cvtColor(edge_img, cv2.COLOR_BGR2RGB))
    axs[0].set_title(title1)
    axs[0].axis("off")
    axs[1].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    axs[1].set_title(title2)
    axs[1].axis("off")
    plt.tight_layout()
    plt.show()


# SELECT IMAGE OR VIDEO (Real Time -> Webcam=True | Detect Video -> Webcam = False)
use_webcam = False
video_path = "assets/video2.mp4"

cap = cv2.VideoCapture(0 if use_webcam else video_path)
if not cap.isOpened():
    raise IOError("Tidak bisa membuka video / webcam.")


# REAL SIZE REFERENCE (cm)
actual_width_cm = 4.0
actual_height_cm = 4.0


# === MAIN LOOP ===
while True:
    ret, img = cap.read()
    if not ret:
        break

    frame = img.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # === 1) OBJECT MASK (bright region only) ===
    _, obj_mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

    # === 2) LOCAL BRIGHTNESS / CONTRAST FIX (only inside detected bright object) ===
    edited = frame.astype(np.float32)
    edited += 100          # brightness +100
    edited = edited * 0.5 + 64   # reduce contrast
    edited = np.clip(edited, 0, 255).astype(np.uint8)

    edited_obj = frame.copy()
    edited_obj[obj_mask == 255] = edited[obj_mask == 255]

    # === 3) EDGE DETECTION PIPELINE (like your improved image version) ===
    gray2 = cv2.cvtColor(edited_obj, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray2, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)
    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # === 4) FIND LARGEST CONTOUR ===
    contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)

        # ✅ ROTATED RECTANGLE (accurate even if object rotates)
        rect = cv2.minAreaRect(largest)
        (cx, cy), (w_rot, h_rot), angle = rect

        detected_width_px = min(w_rot, h_rot)
        detected_height_px = max(w_rot, h_rot)

        # ✅ Accurate cm-per-pixel
        cm_per_pixel_w = actual_width_cm / detected_width_px
        cm_per_pixel_h = actual_height_cm / detected_height_px
        cm_per_pixel_avg = (cm_per_pixel_w + cm_per_pixel_h) / 2

        object_width_cm = detected_width_px * cm_per_pixel_avg
        object_height_cm = detected_height_px * cm_per_pixel_avg

        # Draw bounding box
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)

        cv2.putText(frame, f"Width: {object_width_cm:.2f} cm",
                    (box[0][0], box[0][1] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.putText(frame, f"Height: {object_height_cm:.2f} cm",
                    (box[0][0], box[0][1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Robust Object Measurement (Video)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_21180\2237712310.py:83: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


## Object Measurement using ArUco

In [41]:
import cv2.aruco as aruco
import math
from typing import Optional, Tuple

import cv2, cv2.aruco as aruco
print(cv2.__version__)
print(hasattr(aruco, "detectMarkers"))

4.10.0
False


In [42]:
# Settings (change source if needed)
use_webcam = True
video_path = "assets/video1.mov"  # used if use_webcam == False

# Marker config (lowercase variable name as requested)
marker_size_mm = 50  # mm (50 mm -> 5.0 cm)
target_marker_id = 0
dict_choice = aruco.DICT_4X4_50

# Behavior
mode_soft = True  # if True: use last known scale when marker lost
min_contour_area = 200  # filter out tiny noise contours

In [43]:
def pixel_distance(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    """Return Euclidean distance between two 2D points."""
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

In [44]:
def detect_aruco_scale(
    gray_frame: np.ndarray,
    aruco_detector: aruco.ArucoDetector,
    target_id: int,
    marker_size_cm: float,
) -> Tuple[Optional[float], Optional[float], Optional[np.ndarray]]:
    """
    Detect target ArUco marker and compute cm-per-pixel scale.

    Args:
        gray_frame: Grayscale image.
        aruco_detector: OpenCV ArucoDetector instance.
        target_id: marker id to look for (e.g. 0).
        marker_size_cm: real-world side length of marker in centimeters.

    Returns:
        (cm_per_pixel, measured_marker_cm, corners_array)
        - cm_per_pixel: float or None if not detected
        - measured_marker_cm: measured marker side length in cm (debug), or None
        - corners_array: np.ndarray of 4x2 corner points for the found marker (or None)
    """
    corners, ids, _ = aruco_detector.detectMarkers(gray_frame)

    if ids is None or len(corners) == 0:
        return None, None, None

    ids_flat = ids.flatten()
    if target_id not in ids_flat:
        return None, None, None

    idx = int(np.where(ids_flat == target_id)[0][0])
    c = corners[idx].reshape((4, 2))

    # compute average side length in pixels (top + right) / 2
    top = pixel_distance(c[0], c[1])
    right = pixel_distance(c[1], c[2])
    marker_px = (top + right) / 2.0

    if marker_px <= 0:
        return None, None, None

    cm_per_pixel = marker_size_cm / marker_px
    measured_marker_cm = marker_px * cm_per_pixel  # should equal marker_size_cm (debug)

    return cm_per_pixel, measured_marker_cm, c

In [45]:
def detect_largest_object(gray_frame: np.ndarray) -> Optional[np.ndarray]:
    """
    Detect the largest contour (object) using a Sobel-based pipeline.
    Returns the largest contour or None if not found.
    """
    blur = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)

    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest = max(contours, key=cv2.contourArea)
    if cv2.contourArea(largest) < min_contour_area:
        return None
    return largest

In [46]:
def measure_from_rect(rect: Tuple) -> Tuple[float, float, np.ndarray]:
    """
    Given a minAreaRect result, return (w_px, h_px, box_points)
    where w_px is the smaller side (width) and h_px the larger (height).
    """
    (cx, cy), (w_rot, h_rot), angle = rect
    w_px = min(w_rot, h_rot)
    h_px = max(w_rot, h_rot)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    return w_px, h_px, box

In [47]:
def run_measurement(
    source_is_webcam: bool = True,
    video_path_local: str = "assets/video3",
    marker_size_mm_local: int = 50,
    marker_id_local: int = 0,
    dict_choice_local=aruco.DICT_4X4_50,
    soft_mode: bool = True,
):
    """
    Main loop: open capture, detect ArUco, detect object, measure, and display.

    Args:
        source_is_webcam: True to use webcam (0); False to use video_path_local.
        video_path_local: path to video file if not using webcam.
        marker_size_mm_local: marker physical size in millimeters.
        marker_id_local: ArUco ID to detect.
        dict_choice_local: ArUco dictionary constant.
        soft_mode: if True, reuse last known scale when marker missing.
    """
    marker_size_cm_local = marker_size_mm_local / 10.0

    # Open capture
    cap = cv2.VideoCapture(0 if source_is_webcam else video_path_local)
    if not cap.isOpened():
        raise IOError("Cannot open webcam or video file.")

    # Create ArUco detector (new API)
    aruco_dict_local = aruco.getPredefinedDictionary(dict_choice_local)
    parameters = aruco.DetectorParameters()
    aruco_detector = aruco.ArucoDetector(aruco_dict_local, parameters)

    last_cm_per_pixel_local: Optional[float] = None

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            vis = frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect ArUco and compute scale
            cm_per_pixel, measured_marker_cm, marker_corners = detect_aruco_scale(
                gray, aruco_detector, marker_id_local, marker_size_cm_local
            )

            # If detected, draw blue polygon and label; else fallback
            if marker_corners is not None:
                int_box = np.int0(marker_corners)
                cv2.polylines(vis, [int_box], True, (255, 0, 0), 3)  # blue box

                # display both scale and measured marker size (option C)
                cv2.putText(vis, f"Scale: {cm_per_pixel:.6f} cm/px", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)
                cv2.putText(vis, f"Marker size (meas): {measured_marker_cm:.2f} cm",
                            (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)

                last_cm_per_pixel_local = cm_per_pixel
                marker_status_text = f"Marker OK (ID {marker_id_local})"
            else:
                if last_cm_per_pixel_local is not None and soft_mode:
                    marker_status_text = "Marker lost - using last scale"
                    cv2.putText(vis, f"Scale (last): {last_cm_per_pixel_local:.6f} cm/px",
                                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)
                else:
                    marker_status_text = "No scale - pixel units only"

                # Optionally show that no marker was found
                cv2.putText(vis, marker_status_text, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            # Detect largest object and measure
            largest_contour = detect_largest_object(gray)
            if largest_contour is not None:
                rect = cv2.minAreaRect(largest_contour)
                w_px, h_px, box = measure_from_rect(rect)

                # Draw object bounding box (green)
                cv2.drawContours(vis, [box], 0, (0, 255, 0), 2)

                # Use detected or last-known scale to convert to cm
                used_scale = None
                if cm_per_pixel is not None:
                    used_scale = cm_per_pixel
                elif last_cm_per_pixel_local is not None and soft_mode:
                    used_scale = last_cm_per_pixel_local

                if used_scale is not None:
                    w_cm = w_px * used_scale
                    h_cm = h_px * used_scale
                    cv2.putText(vis, f"W: {w_cm:.2f} cm", (box[0][0], box[0][1] - 15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    cv2.putText(vis, f"H: {h_cm:.2f} cm", (box[0][0], box[0][1] + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                else:
                    # show pixel fallback
                    cv2.putText(vis, f"W: {w_px:.1f} px", (box[0][0], box[0][1] - 15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                    cv2.putText(vis, f"H: {h_px:.1f} px", (box[0][0], box[0][1] + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

            else:
                cv2.putText(vis, "No object contours found", (10, vis.shape[0] - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            # Status text near top-left (marker status)
            cv2.putText(vis, marker_status_text, (10, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 200, 255), 2)

            cv2.imshow("ArUco Measurement (function-based)", vis)

            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()


if __name__ == "__main__":
    # run using the global settings above
    run_measurement(
        source_is_webcam=use_webcam,
        video_path_local=video_path,
        marker_size_mm_local=marker_size_mm,
        marker_id_local=target_marker_id,
        dict_choice_local=dict_choice,
        soft_mode=mode_soft,
    )

C:\Users\lenovo\AppData\Local\Temp\ipykernel_21180\606261165.py:10: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_21180\2252595375.py:50: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  int_box = np.int0(marker_corners)
